In [1]:
# Load the dataset
import numpy as np
import h5py
import sys
sys.path.append ('../src')
train_dataset = h5py.File ('Data/train_catvnoncat.h5','r')
X_train_orig = np.array (train_dataset['train_set_x'][:])
Y_train_orig = np.array (train_dataset['train_set_y'][:])
test_dataset = h5py.File ('Data/test_catvnoncat.h5','r')
X_test_orig = np.array (test_dataset['test_set_x'][:])
Y_test_orig = np.array (test_dataset['test_set_y'][:])

X_train = X_train_orig.reshape (X_train_orig.shape[0],-1) / 255
Y_train = np.array (Y_train_orig).reshape (Y_train_orig.shape[0],1)
X_test = X_test_orig.reshape (X_test_orig.shape[0],-1) / 255
Y_test = np.array (Y_test_orig).reshape (Y_test_orig.shape[0],1)

# Trasponemos la matriz de datos para que coincida con lo que espera la red neuronal de entrada
X_train = X_train.T
Y_train = Y_train.T
X_test = X_test.T
Y_test = Y_test.T


In [2]:
from NeuralNetwork_v2 import SigmoidF,ReluF,TanhF,LReluF,Activation_V2,Linear,Layer,NNClassifier_V2,SoftmaxF, BinaryCrossEntropy, CategoricalCrossEntropy
from sklearn.metrics import multilabel_confusion_matrix,classification_report

layer1 = Layer (128,12288,activationF=ReluF,keep_prob=0.75)
layer2 = Layer (64,128,activationF=ReluF,keep_prob=0.75)
layer3 = Layer (25,64,activationF=ReluF,keep_prob=1.0)
layer4 = Layer (1,25,activationF=SigmoidF,keep_prob=1.0)
layers = [layer1,layer2,layer3,layer4]
gd_dict = {'steps':50,
#           'learning_rate':0.059,
           'learning_rate':1e-4,
           'mini_batch_size':2**8,
           'optimization':'Adam',
           'beta1': 0.9 , #used in momentum y Adam (beta1) (LR=0.059)
           'beta2': 0.999 # used in RMSprop y Adam (LR=1E-4)
          }
nn = NNClassifier_V2 (layers,costFunction=BinaryCrossEntropy,normalization=False,kargs=gd_dict,print_cost=False)
costs=nn.optimize (X_train,Y_train,l2_lambda = 0.0,method='batchGD')
train_prediction = nn.predict (X_train)
train_prediction = train_prediction.T
train_prediction [train_prediction > 0.5 ] = 1
train_prediction [train_prediction <= 0.5 ] = 0
result = train_prediction
y = Y_train.T
accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )


test_predicted = nn.predict (X_test)
test_predicted = test_predicted.T
test_predicted [test_predicted > 0.5 ] = 1
test_predicted [test_predicted <= 0.5 ] = 0
result = test_predicted
y = Y_test.T
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")

LR = 0.0001: 100%|█████████████████████████████████████████████████████████████████████| 50/50 [00:24<00:00,  2.08it/s]

Trainig set accuracy : 71.77033492822966
Test set accuracy : 40.0
Classification report for classifier <NeuralNetwork_v2.NNClassifier_V2 object at 0x0000011C7FA5A5B0>:
              precision    recall  f1-score   support

           0       0.36      0.94      0.52        17
           1       0.80      0.12      0.21        33

    accuracy                           0.40        50
   macro avg       0.58      0.53      0.36        50
weighted avg       0.65      0.40      0.31        50




In [4]:
### import sys
sys.path.append ('../src')

from costs import  BinaryCrossEntropy, CategoricalCrossEntropy
from NeuralNetwork_v3 import NNClassifier_V3
from activationF import SigmoidF,ReluF,TanhF,LReluF,SoftmaxF
from sklearn.metrics import multilabel_confusion_matrix,classification_report
from GD_algorithms import GradientDescent,Momentum,RMSprop,Adam,LRDecay
from layerF import Layer
from sklearn.metrics import multilabel_confusion_matrix,classification_report


layer1 = Layer (128,12288,activationF=ReluF,keep_prob=1.0)
layer2 = Layer (64,128,activationF=ReluF,keep_prob=1.0)
layer3 = Layer (25,64,activationF=ReluF,keep_prob=1.0)
layer4 = Layer (1,25,activationF=SigmoidF,keep_prob=1.0)
layers = [layer1,layer2,layer3,layer4]

gd_dict = {'steps':150,   
           'learning_rate':0.5,
           'decay_rate': 1,
           'mini_batch_size':2**8,
           'beta':0.9,#used in momentum y RMSprop
           'beta1': 0.9 , #used in Adam 
           'beta2': 0.999 # used in Adam 
          }
nn = NNClassifier_V3 (layers,costFunction=BinaryCrossEntropy,normalization=True,optimizationF=LRDecay,kargs=gd_dict,print_cost=False)
costs=nn.train (X_train,Y_train,l2_lambda = 0.0,method='miniBatchGD')
train_prediction = nn.predict (X_train)
train_prediction = train_prediction.T
train_prediction [train_prediction > 0.5 ] = 1
train_prediction [train_prediction <= 0.5 ] = 0
result = train_prediction
y = Y_train.T
accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )


test_predicted = nn.predict (X_test)
test_predicted = test_predicted.T
test_predicted [test_predicted > 0.5 ] = 1
test_predicted [test_predicted <= 0.5 ] = 0
result = test_predicted
y = Y_test.T
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")

LR = 0.5: 100%|██████████████████████████████████████████████████████████████████████| 150/150 [00:33<00:00,  4.42it/s]

Trainig set accuracy : 99.52153110047847
Test set accuracy : 62.0
Classification report for classifier <NeuralNetwork_v3.NNClassifier_V3 object at 0x000001D3E2A8A9D0>:
              precision    recall  f1-score   support

           0       0.47      0.88      0.61        17
           1       0.89      0.48      0.63        33

    accuracy                           0.62        50
   macro avg       0.68      0.68      0.62        50
weighted avg       0.75      0.62      0.62        50




In [5]:
print (costs[-1])

[[0.10052615]]
